In [3]:
import pandas as pd
import xarray as xr
import os

import numpy as np
import gsw
import warnings
from netCDF4 import Dataset
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore", category=DeprecationWarning) 


In [4]:
file = 'processed.nc' 
data = Dataset(file, mode='r') # read the data 
print(type(data)) # print the type of the data 

<class 'netCDF4._netCDF4.Dataset'>


In [ ]:
print(data.variables.keys()) # print the variables in the data

In [ ]:
tem = data.variables['temperature'][:]  
sal = data.variables['salinity'][:]


In [ ]:
del(data)

In [ ]:
# Figure out boudaries (mins and maxs)
smin = sal.min() - (0.01 * sal.min())
smax = sal.max() + (0.01 * sal.max())
tmin = tem.min() - (0.1 * tem.max())
tmax = tem.max() + (0.1 * tem.max())

In [ ]:
# Calculate how many gridcells we need in the x and y dimensions
xdim = int(round((smax-smin)/0.1+1,0))
ydim = int(round((tmax-tmin)+1,0))

# to avoid running into memory space error
# Create temp and salt vectors of appropiate dimensions
ti = np.linspace(1,ydim-1,ydim)+tmin
si = np.linspace(1,xdim-1,xdim)*0.1+smin


In [ ]:
# Create empty grid of zeros
dens = np.zeros((ydim,xdim))

In [ ]:
# Loop to fill in grid with densities
for j in range(0,int(ydim)):
    for i in range(0, int(xdim)):
        dens[j,i]=gsw.rho(si[i],ti[j],0)

In [ ]:
# Substract 1000 to convert to sigma-t
dens = dens - 1000

In [ ]:
fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
CS = plt.contour(si,ti,dens, linestyles='dashed', colors='k')
plt.clabel(CS, fontsize=12, inline=1, fmt='%1.0f') # Label every second level
 
ax1.plot(sal,tem,'or',markersize=9)
 
ax1.set_xlabel('Salinity')
ax1.set_ylabel('Temperature (C)')
plt.show()

In [6]:
# Open the dataset using xarray
file = 'processed.nc'
ds = xr.open_mfdataset(file)

In [ ]:
for dim in ds.dims:
    print(dim)  # list of nc dimensions

In [ ]:
ds.dims['profile']

In [ ]:
ds

In [ ]:
vts = pd.Series(ds['backscatter'].values,index=ds['depth'])
vts

In [7]:
time = ds['time'][:]
time = time.astype(np.float)
pressure = ds['temperature'][:]
temp = ds['salinity'][:]
sal = ds['dissolved_oxygen'][:]

In [9]:
# convert the time variable to datetime
from datetime import datetime
dtime = time/(24*3600) + datetime(1900,1,1).toordinal()
dtime

<xarray.DataArray 'time' (profile: 930)>
dask.array<add, shape=(930,), dtype=float64, chunksize=(930,), chunktype=numpy.ndarray>
Coordinates:
    time       (profile) datetime64[ns] dask.array<chunksize=(930,), meta=np.ndarray>
    latitude   (profile) float64 dask.array<chunksize=(930,), meta=np.ndarray>
    longitude  (profile) float64 dask.array<chunksize=(930,), meta=np.ndarray>
    lat_uv     (profile) float64 dask.array<chunksize=(930,), meta=np.ndarray>
    lon_uv     (profile) float64 dask.array<chunksize=(930,), meta=np.ndarray>
    time_uv    (profile) datetime64[ns] dask.array<chunksize=(930,), meta=np.ndarray>
Dimensions without coordinates: profile

In [ ]:
# Convert dissolvd oxy from micromole/kg to mg/L
ds['dissolved_oxygen'] = ds.dissolved_oxygen * 1.025 / (44.661*0.7)


In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(3,1, sharex=True, figsize=(10,9))
ds.dissolved_oxygen.plot(ax=ax1) 
ds.dissolved_oxygen.plot(ax=ax2) 
ds.dissolved_oxygen.plot(ax=ax3) 


# Add line at 2mg/L for dissolved_oxygen

xlim = ax2.get_xlim()
ax2.plot([xlim[0],xlim[1]],[2,2],'r--')

# Add 0 line for Wind
ax3.plot([xlim[0],xlim[1]],[0,0],'k')
ax3.set_xlim(xlim);

In [ ]:
xr.plot.line(ds['temperature'],ds['salinity'])